In [10]:
#import sqlite3
import sqlite3

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
#create the connections of data base
conn = sqlite3.connect('test.db')

In [13]:
cursor = conn.cursor()


In [14]:
#create table
sql = "CREATE TABLE covid2 (Ind_ID int,Test_date date,Cough_symptoms varchar(127),Fever varchar(127),Sore_throat varchar(127),Shortness_of_breath varchar(127),Headache varchar(127),Corona varchar(127),Age_60_above varchar(127),Gender varchar(127),Known_contact varchar(127))"
cursor.execute(sql)
conn.commit()
conn.close()

In [15]:
import pandas as pd

In [16]:
#read file
df = pd.read_csv('/content/drive/MyDrive/covid1.csv')

In [17]:
#inserting values

conn = sqlite3.connect('test.db')
cursor = conn.cursor()

cols = ",".join(df.columns)

# Iterate over the rows of the dataframe
for i, row in df.iterrows():
    # Create the values string for the insert statement
    values = ",".join(["'" + str(x) + "'" for x in row.tolist()])
    
    # Execute the insert statement
    sql = "INSERT INTO covid2 ({}) VALUES ({})".format(cols, values)
    cursor.execute(sql)
    conn.commit()

# Close the cursor and connection
cursor.close()
conn.close()

**1. Find the number of corona patients who faced shortness of breath.**

In [18]:
conn = sqlite3.connect('test.db')
cursor = conn.cursor()

sql = "select count(*) from covid2 where corona = 'positive' and Shortness_of_breath = 'True'"

cursor.execute(sql)

results = cursor.fetchall()

for row in results:
    print(row)

# Close the connection when you're done
conn.close()

(1162,)


**2.Find the number of negative corona patients who have fever and sore_throat.**

In [45]:
conn = sqlite3.connect('test.db')
cursor = conn.cursor()

sql =  "select count(*) from covid2 where corona = 'negative' and Fever = 'true' and Sore_throat = 'true'"

cursor.execute(sql)

results = cursor.fetchall()

for row in results:
    print(row)

# Close the connection when you're done
conn.close()

(0,)


3. **Group the data by month and rank the number of positive cases.**

In [47]:
conn = sqlite3.connect('test.db')
cursor = conn.cursor()

sql = "select strftime('%m', date(Test_date)) as Month, count(*) as counts from (select*from covid2 where corona = 'positive')xyz GROUP BY Month order by counts desc"

 

cursor.execute(sql)

results = cursor.fetchall()

for row in results:
    print(row)

# Close the connection when you're done
conn.close()

('04', 8862)
('03', 5832)


4.**Find the female negative corona patients who faced cough and headache.**

In [33]:
conn = sqlite3.connect('test.db')
cursor = conn.cursor()

sql = "select*from covid2 where corona = 'negative' and Cough_symptoms = 'True' and Headache = 'True' and gender = 'female'"

cursor.execute(sql)

results = cursor.fetchall()

for row in results:
    print(row)

# Close the connection when you're done
conn.close()

(13756, '2020-03-22', 'True', 'True', 'True', 'False', 'True', 'negative', 'No', 'female', 'Abroad')
(17289, '2020-03-22', 'True', 'True', 'True', 'False', 'True', 'negative', 'No', 'female', 'Abroad')
(17657, '2020-03-23', 'True', 'False', 'True', 'False', 'True', 'negative', 'No', 'female', 'Abroad')
(19554, '2020-03-23', 'True', 'True', 'False', 'False', 'True', 'negative', 'No', 'female', 'Other')
(19615, '2020-03-23', 'True', 'False', 'True', 'True', 'True', 'negative', 'No', 'female', 'Contact with confirmed')
(20248, '2020-03-23', 'True', 'True', 'False', 'False', 'True', 'negative', 'Yes', 'female', 'Abroad')
(20253, '2020-03-23', 'True', 'True', 'False', 'False', 'True', 'negative', 'No', 'female', 'Contact with confirmed')
(37904, '2020-03-27', 'True', 'True', 'True', 'True', 'True', 'negative', 'No', 'female', 'Contact with confirmed')
(40616, '2020-03-27', 'True', 'False', 'False', 'True', 'True', 'negative', 'No', 'female', 'Contact with confirmed')
(40752, '2020-03-27', '

5.**How many elderly corona patients have faced breathing problems?**

In [50]:
conn = sqlite3.connect('test.db')
cursor = conn.cursor()

sql = "select count(*) from covid2 where corona = 'positive' and  Shortness_of_breath = 'True' and Age_60_above = 'Yes'"

cursor.execute(sql)

results = cursor.fetchall()

for row in results:
    print(row)

# Close the connection when you're done
conn.close()

(262,)


6. **Which three symptoms were more common among COVID positive patients?**


In [43]:
conn = sqlite3.connect('test.db')
cursor = conn.cursor()

sql = sql = """ select * from (
    select cough_symptoms , count(*) as cnt from covid2 where corona = 'positive' and cough_symptoms ='True'
    union all
    select  Fever, count(*) as cnt from covid2 where corona = 'positive' and Fever ='True'
    union all
    select  Sore_throat, count(*) as cnt from covid2 where corona = 'positive' and Sore_throat ='True'
    union all
    select  Shortness_of_breath, count(*) as cnt from covid2 where corona = 'positive' and Shortness_of_breath ='True'
    union all
select  Headache, count(*) as cnt from covid2 where corona = 'positive' and Headache ='True'
   ) xyz order by cnt desc

"""

cursor.execute(sql)

results = cursor.fetchall()

for row in results:
    print(row)

# Close the connection when you're done
conn.close()

('True', 6582)
('True', 5558)
('True', 2230)
('True', 1523)
('True', 1162)


7. **Which symptom was less common among COVID negative people?**

In [52]:
conn = sqlite3.connect('test.db')
cursor = conn.cursor()

sql = """ select * from (
    select cough_symptoms , count(*) as cnt from covid2 where corona = 'negative' and cough_symptoms ='True'
    union all
    select  Fever, count(*) as cnt from covid2 where corona = 'negative' and Fever ='True'
    union all
    select  Sore_throat, count(*) as cnt from covid2 where corona = 'negative' and Sore_throat ='True'
    union all
    select  Shortness_of_breath, count(*) as cnt from covid2 where corona = 'negative' and Shortness_of_breath ='True'
    union all
    select  Headache, count(*) as cnt from covid2 where corona = 'negative' and Headache ='True'
  
    ) xyz order by cnt
"""

cursor.execute(sql)

results = cursor.fetchall()

for row in results:
    print(row)

# Close the connection when you're done
conn.close()

('True', 147)
('True', 365)
('True', 384)
('True', 15816)
('True', 34987)


8. **What are the most common symptoms among COVID positive males whose known contact was abroad?** 

In [54]:
conn = sqlite3.connect('test.db')
cursor = conn.cursor()

sql =  """
    select cough_symptoms , count(*) as cnt from covid2 
    where corona = 'positive' and cough_symptoms ='True' 
    and gender = 'male' and known_contact = 'Abroad'
    union all
    select  Fever, count(*) as cnt from covid2 
    where corona = 'positive' and Fever ='True'
    and gender = 'male' and known_contact = 'Abroad'
    union all
    select  Sore_throat, count(*) as cnt from covid2 
    where corona = 'positive' and Sore_throat ='True'
    and gender = 'male' and known_contact = 'Abroad'
    union all
    select  Shortness_of_breath, count(*) as cnt from covid2 
    where corona = 'positive' and Shortness_of_breath ='True'
    and gender = 'male' and known_contact = 'Abroad'
    union all
    select  Headache, count(*) as cnt from covid2
    where corona = 'positive' and Headache ='True'
    and gender = 'male' and known_contact = 'Abroad'
   

"""

cursor.execute(sql)

results = cursor.fetchall()

for row in results:
    print(row)

# Close the connection when you're done
conn.close()

('True', 739)
('True', 608)
('True', 155)
('True', 150)
('True', 209)
